##**Ranking nodes with degree centrality**

In this Jupyter notebook instance, we simplify our model to single layer and test its effectiveness of approximating degree centrality ranking with varying graph structures. Results show that the model below can rank nodes based on degree centrality without actually getting any explicit information about degree in test datasets.


Graphs used are undirected random graphs with 6 types of variants from networkx. 4 types are used for training and 2 types for testing. Though the ratio can be changed to see the effects. Number of epoches can be increased if the number of training graphs are being reduced.

For choice of graphs, I just chose ones which can be generated by providing simply number of nodes, number of edges and minimum or no parameters.

Note 1: The method doesn't work so well with Watts_Strogatz graphs, as you can check that there are many nodes with same degree values. The reason I think is that many nodes have same degree centrality values so model is not able to distinguish their ranking and it becomes harder to rank them.

Note 2 : This Jupyter notebook was tested on Google Colaboratory.



In [0]:
#Import stuff

import networkx as nx
import numpy as np
import torch
import torch.nn as nn
import time
from scipy.stats import kendalltau
import random
random.seed(10)

In [0]:

def get_graph(num_nodes,num_edges,index):
  """Generate and return different types of graphs."""
  
  if index==0:
    g = nx.gnm_random_graph(num_nodes,num_edges)
    return g
  
  elif index==1:
    g = nx.barabasi_albert_graph(num_nodes,num_nodes-1)
    return g
  
  elif index==2:
    g = nx.powerlaw_cluster_graph(num_nodes,num_nodes-1,0.2)
    return g
  
  elif index==3:
    p = (num_edges/(num_nodes*num_nodes))
    g = nx.fast_gnp_random_graph(num_nodes,p)
    return g
  
  elif index==4:
    #parameters taken from NetworkX example
    g = nx.gaussian_random_partition_graph(100,10,10,.25,.1,directed=False)
    return nx.Graph(g)
  
  elif index==5:
    g = nx.watts_strogatz_graph(num_nodes,3,0.1)
    return g
  
  
def get_adjacency(list_graph,num_nodes):
  """Returns adjacency matrices, their node sequences and centrality matrix."""
  num_graph = len(list_graph)
  list_adj = list()
  cent_mat = np.zeros((num_nodes,num_graph),dtype=np.float)
  list_nodelist = list()
  
  
  for ind,g in enumerate(list_graph):
    node_seq = list(g.nodes())
    random.shuffle(node_seq)
    degree_cent = nx.degree_centrality(g)
    assert len(degree_cent)==100,"Number of nodes do not match"
    
    for i in range(num_nodes):
      cent_mat[i,ind] = degree_cent[node_seq[i]]
    
    adj_mat = nx.adjacency_matrix(g,nodelist=node_seq)
    adj_mat = sparse_mx_to_torch_sparse_tensor(adj_mat)
    list_adj.append(adj_mat)
    list_nodelist.append(node_seq)
    
       
  return list_adj,list_nodelist,cent_mat
      
   
def loss_cal(y_out,true_val,device):
    """Calculates ranking based loss function."""
    #Here we check the pairwise ranking of nodes. As including all pair of nodes
    #is going to be computationally expensive, I chose to select N*20 number of node pairs.
  
    model_size = y_out.shape[0]
    num_nodes = model_size

    y_out = y_out.reshape((model_size))
    true_val = true_val.reshape((model_size))
    
    _,order_y_true = torch.sort(-true_val[:num_nodes])

    sample_num = num_nodes*20

    ind_1 = torch.randint(0,num_nodes,(sample_num,)).long().to(device)
    ind_2 = torch.randint(0,num_nodes,(sample_num,)).long().to(device)
    

    rank_measure=torch.sign(-1*(ind_1-ind_2)).float()
        
    input_arr1 = y_out[:num_nodes][order_y_true[ind_1]].to(device)
    input_arr2 = y_out[:num_nodes][order_y_true[ind_2]].to(device)
        

    loss_rank = torch.nn.MarginRankingLoss(margin=1.0).forward(input_arr1,input_arr2,rank_measure)
 
    return loss_rank


def ranking_correlation(y_out,true_val):
    """Returns Kendall-Tau's correlation and top-k accuracy."""
    model_size = y_out.shape[0]
    node_num = model_size
    y_out = y_out.reshape((model_size))
    true_val = true_val.reshape((model_size))

    predict_arr = y_out.cpu().detach().numpy()
    true_arr = true_val.cpu().detach().numpy()
    
    #check top-k accuracy
    k = 20
    top_k_predict = predict_arr.argsort()[::-1][:k]
    top_k_true = true_arr.argsort()[::-1][:k]
    
    acc = 0
    for item in top_k_predict:
      if item in top_k_true:
        acc += 1
        
    kt,_ = kendalltau(predict_arr[:node_num],true_arr[:node_num])

    return kt, (acc/k)*100,k

def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)
  
  
    
def train(list_adj,cent_mat,device):
  """Training the model."""
  model.train()
  num_train = len(list_adj)
  for i in range(num_train):
    adj = list_adj[i]
    adj = adj.to(device)
    
    optimizer.zero_grad()
    y_out = model(adj)
    true_val = torch.from_numpy(cent_mat[:,i]).float()
    true_val = true_val.to(device)
    
    loss_rank = loss_cal(y_out,true_val,device)
    loss_rank.backward()
    optimizer.step()
    
    
def test(list_adj,cent_mat,device):
  """Testing the model."""
  model.eval()
  list_kt = list()
  list_acc = list()
  num_test = len(list_adj)
  for j in range(num_test):
    adj = list_adj[j]
    adj = adj.to(device)
    
    y_out = model(adj)
    
    true_val = torch.from_numpy(cent_mat[:,j]).float()
    true_val = true_val.to(device)
    
    kt,acc,k = ranking_correlation(y_out,true_val)
    list_kt.append(kt)
    list_acc.append(acc)
    
  print(f"Average Kendall Tau score is: {np.mean(np.array(list_kt))} and std:{np.std(np.array(list_kt))}")
  print(f"Average top-{k} accuracy:{np.mean(list_acc)}% and std:{np.std(list_acc)}")
  
  

In [32]:
#generate the indices for graph generation
#select all graphs and train/test on different graphs
graph_index = list(range(6))

#To generate graphs of same type,comment the above line and uncomment the line below
# index in bracket[] decides which graph to choose
#graph_index = [4]*6

#shuffle the indices for random training and testing dataset
#Running this every time will create different combinations

random.shuffle(graph_index)

#choose 4 graph variants for training and 2 for testing
train_index = graph_index[:4]
test_index = graph_index[4:]

#create list of graphs for training
#below numbers are chosen randomly without much thought
num_nodes = 100
num_edges = 400

list_train_graph = list()
list_test_graph = list()

#create list for training graphs
print("Create training graphs")
for i in range(50):
  for j in train_index:
    new_graph = get_graph(num_nodes,num_edges,j)
    
    #making sure graphs were generated as we wanted
    assert nx.is_directed(new_graph)==False,"Graph is directed"
    assert nx.number_of_selfloops(new_graph)==0,"Graph has self-loops"
    assert new_graph.number_of_nodes() == num_nodes,"Number of nodes are higher"
    
    list_train_graph.append(new_graph)
    
#create list for test graphs
print("Create testing graphs")
for i in range(5):
  for j in test_index:
    new_graph = get_graph(num_nodes,num_edges,j)
    
    #making sure graphs were generated as we wanted
    assert nx.is_directed(new_graph)==False,"Graph is directed"
    assert nx.number_of_selfloops(new_graph)==0,"Graph has self-loops"
    list_test_graph.append(new_graph)

#shuffle the graphs
random.shuffle(list_train_graph)
random.shuffle(list_test_graph)


#get training and testing data

list_adj_train,list_nodelist_train,cent_mat_train = get_adjacency(list_train_graph,num_nodes)
list_adj_test,list_nodelist_test,cent_mat_test = get_adjacency(list_test_graph,num_nodes)







    



Create training graphs
Create testing graphs


In [0]:
#define simple MLP module
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size,hidden_size)
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size, output_size)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_size = num_nodes
hidden_size = 16
output_size = 1
model = NeuralNet(input_size, hidden_size, output_size).to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=0.0005)


In [34]:
#Training the model 
#To check for different combinations of graphs its better to just click on 
# "Restart and Run all" in Runtime menu.

num_epoch = 10

for e in range(num_epoch):
  print("--------------------------------------------------------")
  print(f"Epoch number: {e}")
  train(list_adj_train,cent_mat_train,device)
  test(list_adj_test,cent_mat_test,device)
  


--------------------------------------------------------
Epoch number: 0
Average Kendall Tau score is: 0.7853640460813041 and std:0.02905096230478589
Average top-20 accuracy:75.0% and std:7.0710678118654755
--------------------------------------------------------
Epoch number: 1
Average Kendall Tau score is: 0.9193699863927408 and std:0.00925319424462081
Average top-20 accuracy:87.5% and std:3.3541019662496847
--------------------------------------------------------
Epoch number: 2
Average Kendall Tau score is: 0.9534167747306178 and std:0.006485969252174808
Average top-20 accuracy:92.5% and std:4.6097722286464435
--------------------------------------------------------
Epoch number: 3
Average Kendall Tau score is: 0.9559727914090107 and std:0.008695863686838983
Average top-20 accuracy:93.0% and std:4.58257569495584
--------------------------------------------------------
Epoch number: 4
Average Kendall Tau score is: 0.955931059485191 and std:0.00863798621454829
Average top-20 accuracy